# 🔄 통합 소스 파이프라인 노트북

이 노트북은 **3단계 파이프라인**을 하나로 통합하여 실행합니다:

```
Generator → Validator → Indexer
```

## 3단계 파이프라인 개요

### 📝 PHASE 1: 소스 생성 (Generator)
- LLM의 내장 지식을 활용하여 5가지 원본 소스 자동 생성
- 입력: shop_seq, shop_name, category
- 출력: collected_data (메모리)

### ✅ PHASE 2: 데이터 검증 (Validator)
- 생성된 소스 데이터의 유효성 검증
- 필수 필드, 데이터 타입, 텍스트 길이 확인
- 출력: 검증된 collected_data (메모리)

### 🗄️ PHASE 3: 벡터 DB 인덱싱 (Indexer)
- 소스 타입별 청킹 전략 적용
- 768차원 임베딩 생성
- Chroma 벡터 DB에 저장 (카테고리별 독립 컬렉션)

## 장점

✅ **단일 실행**: 한 번에 전체 파이프라인 실행  
✅ **명확한 구분**: 3단계가 시각적으로 명확하게 구분됨  
✅ **메모리 공유**: JSON 파일 I/O 없이 메모리에서 데이터 전달  
✅ **유지보수 용이**: 한 파일에서 모든 로직 관리  

## 사용 방법

1. 섹션 5에서 매장 리스트 입력 (`BATCH_SHOPS`)
2. 순차적으로 모든 셀 실행 (Run All)
3. 각 Phase의 결과 확인
4. 최종 벡터 DB에 인덱싱 완료

## 섹션 1: 패키지 설치

In [ ]:
%pip install google-cloud-aiplatform google-genai python-dotenv chromadb langchain langchain-text-splitters pandas

## 섹션 2: 라이브러리 Import

In [ ]:
import os
import json
import time
import pandas as pd
import numpy as np
import vertexai
from google import genai
from google.genai import types
from dotenv import load_dotenv
from datetime import datetime
import chromadb
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List, Dict

print("✅ 라이브러리 Import 완료")

## 섹션 3: Vertex AI 초기화

In [ ]:
load_dotenv()

PROJECT_ID = "wad-dw"
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Vertex AI 초기화 완료: {PROJECT_ID} ({LOCATION})")

---

# 📝 PHASE 1: 소스 생성 (Generator)

LLM의 내장 지식을 활용하여 5가지 원본 소스를 자동 생성합니다.

---

## 섹션 4: LLM 지식 기반 수집 함수

In [ ]:
# ========================================
# LLM 지식 기반 수집
# ========================================

MODEL_NAME = "gemini-2.5-pro"

print("✅ LLM 지식 기반 모드 초기화 완료")
print(f"   - 모델: {MODEL_NAME}")
print(f"   - 수집 방법: LLM 내장 지식만 사용")
print(f"   - 비용: 토큰 비용만 (~$0.02/매장)")


# ========================================
# 카테고리별 소스 타입 정의
# ========================================

CATEGORY_SOURCE_CONFIGS = {
    "fine_dining": {
        "michelin_review": {
            "name": "미쉐린 가이드",
            "prompt": """이 레스토랑에 대한 미쉐린 가이드 정보를 요약하세요:
            - 미쉐린 별점 (0-3스타, 빕 구르망 등)
            - 가격대
            - 주요 평가 내용
            - 레스토랑 특징

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        },
        "blueribbon_review": {
            "name": "블루리본 서베이",
            "prompt": """이 레스토랑에 대한 블루리본 서베이 정보를 요약하세요:
            - 블루리본 등급/점수
            - 평가 내용
            - 레스토랑 특징

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        },
        "chef_interview": {
            "name": "셰프 인터뷰",
            "prompt": """이 레스토랑 셰프에 대한 정보를 요약하세요:
            - 셰프 이름 및 경력
            - 요리 철학
            - 식재료 활용 방식
            - 조리 기법 및 스타일

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        },
        "course_description": {
            "name": "코스 구성",
            "prompt": """이 레스토랑의 메뉴 정보를 요약하세요:
            - 코스 구성 (가능하면 구체적으로)
            - 시그니처 메뉴
            - 대표 요리
            - 가격대

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        },
        "brand_philosophy": {
            "name": "철학/공간/전통",
            "prompt": """이 레스토랑의 브랜드 정보를 요약하세요:
            - 브랜드 철학 및 스토리
            - 공간 디자인 및 분위기
            - 인테리어 특징
            - 전통 기법 활용

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        }
    },
    
    "waiting_hotplace": {
        "signature_menu": {
            "name": "시그니처 메뉴",
            "prompt": """이 레스토랑의 시그니처 메뉴와 인기 메뉴를 요약하세요:
            - 대표 메뉴 및 시그니처 요리
            - 메뉴별 특징과 인기 이유
            - SNS에서 화제가 된 메뉴
            - 추천 메뉴 조합

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        },
        "atmosphere": {
            "name": "분위기/공간",
            "prompt": """이 레스토랑의 분위기와 공간 정보를 요약하세요:
            - 인테리어 스타일과 분위기
            - 좌석 배치 및 공간 구성
            - 인스타그래머블한 포인트
            - 방문객 연령대 및 목적

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        },
        "popularity": {
            "name": "인기도/웨이팅",
            "prompt": """이 레스토랑의 인기도와 웨이팅 정보를 요약하세요:
            - 평균 웨이팅 시간
            - 피크 시간대 및 한적한 시간대
            - SNS 언급량 및 리뷰 반응
            - 인기 비결과 핫플레이스 요소

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        },
        "price_value": {
            "name": "가격/가성비",
            "prompt": """이 레스토랑의 가격대와 가성비 정보를 요약하세요:
            - 주요 메뉴 가격대
            - 1인당 평균 예산
            - 가성비 평가 및 가격 대비 만족도
            - 추가 비용 (서비스 차지, 음료 등)

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        },
        "location_access": {
            "name": "위치/접근성",
            "prompt": """이 레스토랑의 위치와 접근성 정보를 요약하세요:
            - 정확한 위치 및 주변 랜드마크
            - 대중교통 접근성 (지하철역, 버스 등)
            - 주차 가능 여부 및 주차 정보
            - 주변 관광지나 함께 방문하기 좋은 장소

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        }
    },
    
    "mid_to_low": {
        "menu_composition": {
            "name": "메뉴 구성",
            "prompt": """이 레스토랑의 메뉴 구성을 요약하세요:
            - 메뉴 카테고리 및 구성
            - 코스 메뉴 또는 단품 메뉴 정보
            - 계절 메뉴 및 정기 변경 여부
            - 메뉴 다양성 및 선택 폭

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        },
        "value_proposition": {
            "name": "가성비/가치",
            "prompt": """이 레스토랑의 가성비와 가치를 요약하세요:
            - 가격대 (1인당 평균 비용)
            - 가격 대비 품질 및 양
            - 예약 고객 대상 혜택이나 특별 메뉴
            - 가성비 포인트 및 추천 이유

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        },
        "dining_atmosphere": {
            "name": "분위기/경험",
            "prompt": """이 레스토랑의 식사 분위기와 경험을 요약하세요:
            - 레스토랑 분위기 (캐주얼, 격식, 로맨틱 등)
            - 적합한 방문 목적 (데이트, 가족 모임, 비즈니스 등)
            - 서비스 수준 및 특징
            - 특별한 경험 요소

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        },
        "reservation_parking": {
            "name": "예약/주차",
            "prompt": """이 레스토랑의 예약과 주차 정보를 요약하세요:
            - 예약 방법 및 난이도
            - 예약 필수 여부 및 당일 예약 가능 여부
            - 주차 시설 (발렛, 자체 주차장, 주변 주차장)
            - 예약 시 유의사항

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        },
        "chef_approach": {
            "name": "셰프 접근법",
            "prompt": """이 레스토랑 셰프의 요리 접근법을 요약하세요:
            - 셰프의 배경 및 경력 (간략히)
            - 요리 스타일 및 특징
            - 식재료 선택 및 활용 방식
            - 셰프의 철학이 메뉴에 반영된 부분

            300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
        }
    }
}


# ========================================
# LLM 지식 기반 수집 함수 (재시도 로직 포함)
# ========================================

def collect_with_llm_knowledge(shop_name: str, source_type: str, config: dict, max_retries: int = 2) -> dict:
    """LLM의 내장 지식을 사용하여 정보 수집 (자동 재시도 포함)"""
    
    for attempt in range(max_retries):
        try:
            full_prompt = f"""
            매장명: {shop_name}

            당신의 학습된 지식을 바탕으로 다음 작업을 수행하세요:

            {config['prompt']}

            중요:
            - 학습된 정보만 사용하세요
            - 확실한 내용만 포함하세요
            - 추측하지 마세요
            - 정보가 없거나 확실하지 않으면 found를 false로 설정하세요
            - JSON 문자열 내부의 따옴표는 반드시 백슬래시로 이스케이프하세요 (예: \\" )
            - 줄바꿈은 \\n으로 표현하세요

            출력 형식 (JSON):
            {{
            "text": "내용...",
            "found": true
            }}
            """
            
            response = client.models.generate_content(
                model=MODEL_NAME,
                contents=full_prompt,
                config=types.GenerateContentConfig(
                    temperature=0.3,
                    max_output_tokens=2048,
                    response_mime_type="application/json"
                )
            )
            
            # JSON 파싱 시도
            result = json.loads(response.text)
            
            if result.get("found") and result.get("text"):
                return {
                    "success": True,
                    "text": result["text"],
                    "url": "",
                    "method": "llm_knowledge"
                }
            else:
                return {
                    "success": False,
                    "reason": "LLM이 확실한 정보를 제공할 수 없음"
                }
                
        except json.JSONDecodeError as e:
            if attempt < max_retries - 1:
                # 재시도
                time.sleep(1)
                continue
            else:
                return {
                    "success": False,
                    "reason": f"JSON 파싱 오류 (재시도 {max_retries}회 실패): {str(e)}"
                }
        except Exception as e:
            if attempt < max_retries - 1:
                # 재시도
                time.sleep(1)
                continue
            else:
                return {
                    "success": False,
                    "reason": f"LLM 오류 (재시도 {max_retries}회 실패): {str(e)}"
                }
    
    return {
        "success": False,
        "reason": "알 수 없는 오류"
    }


# ========================================
# 매장별 자동 수집 (카테고리별 소스)
# ========================================

def auto_collect_shop_sources(shop_seq: int, shop_name: str, category: str) -> dict:
    """한 매장에 대해 카테고리별 소스를 LLM 지식으로 수집"""
    print(f"\n🔍 {shop_name} 정보 수집 중 (LLM 지식)...")
    print("─" * 80)
    
    # 카테고리에 맞는 소스 설정 가져오기
    source_configs = CATEGORY_SOURCE_CONFIGS.get(category)
    
    if not source_configs:
        print(f"   ❌ 유효하지 않은 카테고리: {category}")
        print(f"   지원 카테고리: {', '.join(CATEGORY_SOURCE_CONFIGS.keys())}")
        return {
            "shop_seq": shop_seq,
            "shop_name": shop_name,
            "category": category,
            "sources": []
        }
    
    sources = []
    
    for source_type, config in source_configs.items():
        print(f"   [{config['name']}]...", end=" ")
        
        result = collect_with_llm_knowledge(shop_name, source_type, config)
        
        if result["success"]:
            sources.append({
                "source_type": source_type,
                "text": result["text"],
                "url": result.get("url", ""),
                "collected_date": datetime.now().strftime("%Y-%m-%d"),
                "collection_method": "llm_knowledge"
            })
            print(f"✅ ({len(result['text'])}자)")
        else:
            print(f"⚠️ {result['reason']}")
        
        time.sleep(0.5)
    
    return {
        "shop_seq": shop_seq,
        "shop_name": shop_name,
        "category": category,
        "sources": sources
    }


print("\n✅ LLM 지식 기반 수집 함수 정의 완료")
print(f"   - 카테고리별 소스 타입:")
for category, sources in CATEGORY_SOURCE_CONFIGS.items():
    print(f"     • {category}: {len(sources)}가지 소스")
print(f"   - 모두 LLM 내장 지식 사용")
print(f"   - 자동 재시도: 최대 2회")

## 섹션 5: 매장 리스트 입력

수집할 매장의 shop_seq, shop_name, category를 입력하세요.

In [ ]:
# ========================================
# 매장 리스트 입력
# ========================================

# 수집할 매장 리스트 (shop_seq + shop_name + category)
BATCH_SHOPS = [
    {"shop_seq": 1, "shop_name": "모수", "category": "fine_dining"},
    {"shop_seq": 2, "shop_name": "밍글스", "category": "fine_dining"},
    {"shop_seq": 3, "shop_name": "스시 소라", "category": "fine_dining"},
    # 추가 매장을 여기에 입력하세요...
]

print(f"📋 자동 수집 대상 매장")
print("=" * 80)
print(f"\n총 {len(BATCH_SHOPS)}개 매장:\n")

for idx, shop in enumerate(BATCH_SHOPS, 1):
    print(f"   [{idx}] {shop['shop_name']} (seq: {shop['shop_seq']}, category: {shop['category']})")

print(f"\n카테고리별 자동 수집 소스 타입 (LLM 지식):")
print("─" * 80)

for category, source_configs in CATEGORY_SOURCE_CONFIGS.items():
    print(f"\n📂 {category} ({len(source_configs)}가지):")
    for source_type, config in source_configs.items():
        print(f"   • {config['name']} ({source_type})")

print("\n" + "=" * 80)

## 섹션 6: LLM 지식 기반 자동 수집 실행

⚠️ LLM의 내장 지식만 사용하여 5가지 소스를 자동 생성합니다.

In [ ]:
# ========================================
# LLM 지식 기반 자동 수집 실행
# ========================================

print("🚀 LLM 지식 기반 자동 수집 시작\n")
print("=" * 80)
print("📚 모든 소스: LLM 내장 지식 활용")
print("⚡ 빠른 처리: 10-15초/매장")
print("💰 저렴한 비용: ~$0.02/매장")
print("=" * 80)

collected_data = []
collection_stats = []

start_time = time.time()

for idx, shop in enumerate(BATCH_SHOPS, 1):
    print(f"\n[{idx}/{len(BATCH_SHOPS)}] {shop['shop_name']} ({shop['category']})")
    print("=" * 80)
    
    shop_start = time.time()
    
    try:
        # LLM 지식 기반 수집 실행
        result = auto_collect_shop_sources(
            shop_seq=shop["shop_seq"],
            shop_name=shop["shop_name"],
            category=shop["category"]
        )
        
        shop_elapsed = time.time() - shop_start
        
        # 통계
        num_sources = len(result.get("sources", []))
        total_chars = sum(len(s["text"]) for s in result.get("sources", []))
        
        print(f"\n✅ 완료: {num_sources}/5개 소스 수집 ({shop_elapsed:.1f}초)")
        print(f"   - 카테고리: {result.get('category')}")
        print(f"   - 총 텍스트: {total_chars:,}자")
        
        collected_data.append(result)
        collection_stats.append({
            "shop_name": shop["shop_name"],
            "category": shop["category"],
            "status": "success",
            "num_sources": num_sources,
            "total_chars": total_chars,
            "elapsed_time": shop_elapsed
        })
        
    except Exception as e:
        print(f"\n❌ 오류: {e}")
        collection_stats.append({
            "shop_name": shop["shop_name"],
            "category": shop.get("category", "unknown"),
            "status": "error",
            "error": str(e)
        })
    
    # 다음 매장까지 대기
    if idx < len(BATCH_SHOPS):
        wait_seconds = 2
        print(f"\n⏳ 다음 매장까지 {wait_seconds}초 대기...")
        time.sleep(wait_seconds)

total_elapsed = time.time() - start_time

print("\n" + "=" * 80)
print(f"\n✅ PHASE 1 완료: 소스 생성 (총 {total_elapsed:.1f}초 = {total_elapsed/60:.1f}분)\n")

# 통계
success_count = sum(1 for s in collection_stats if s.get("status") == "success")
total_sources = sum(s.get("num_sources", 0) for s in collection_stats)

print(f"📊 통계:")
print(f"   - 성공: {success_count}/{len(BATCH_SHOPS)}개 매장")
print(f"   - 총 소스: {total_sources}개 (평균 {total_sources/success_count if success_count > 0 else 0:.1f}개/매장)")
print(f"   - 평균 처리 시간: {total_elapsed / len(BATCH_SHOPS):.1f}초/매장")

# 비용 계산
llm_tokens_estimate = total_sources * 2000  # 소스당 약 2K tokens
cost_estimate = (llm_tokens_estimate / 1000000) * 1.25  # Input token cost for Gemini 2.5 Pro
print(f"\n💰 예상 비용:")
print(f"   - LLM 토큰: 약 ${cost_estimate:.3f}")
print(f"   - **총 비용: 약 ${cost_estimate:.3f}** (매장당 ${cost_estimate/success_count if success_count > 0 else 0:.3f})")

print("\n" + "=" * 80)
print("\n📌 다음 단계: PHASE 2 - 데이터 검증")

---

# ✅ PHASE 2: 데이터 검증 (Validator)

생성된 소스 데이터의 유효성을 검증합니다.

---

## 섹션 7: 데이터 스키마 정의

In [ ]:
# ========================================
# 데이터 스키마
# ========================================

VALID_SOURCE_TYPES = [
    # fine_dining 소스
    "michelin_review",
    "blueribbon_review",
    "chef_interview",
    "course_description",
    "brand_philosophy",
    # waiting_hotplace 소스
    "signature_menu",
    "atmosphere",
    "popularity",
    "price_value",
    "location_access",
    # low_to_mid_price_dining 소스
    "menu_composition",
    "value_proposition",
    "dining_atmosphere",
    "reservation_parking",
    "chef_approach",
    # 레거시 (이전 버전 호환)
    "space_ambiance",
    "tradition_technique"
]

VALID_CATEGORIES = [
    "fine_dining",
    "mid_price",
    "waiting_hotplace"
]

def validate_source_data(data: Dict) -> tuple[bool, str]:
    """
    소스 데이터 유효성 검증
    
    Returns:
        (is_valid, error_message)
    """
    # 필수 필드 확인
    required_fields = ["shop_seq", "shop_name", "category", "sources"]
    for field in required_fields:
        if field not in data:
            return False, f"필수 필드 누락: {field}"
    
    # 카테고리 확인
    if data["category"] not in VALID_CATEGORIES:
        return False, f"유효하지 않은 카테고리: {data['category']}"
    
    # sources 리스트 확인
    if not isinstance(data["sources"], list):
        return False, "sources는 리스트여야 합니다"
    
    if len(data["sources"]) == 0:
        return False, "최소 1개 이상의 소스가 필요합니다"
    
    # 각 소스 검증
    for idx, source in enumerate(data["sources"]):
        # 필수 필드
        source_required = ["source_type", "text"]
        for field in source_required:
            if field not in source:
                return False, f"소스 {idx}번: 필수 필드 누락: {field}"
        
        # source_type 확인
        if source["source_type"] not in VALID_SOURCE_TYPES:
            return False, f"소스 {idx}번: 유효하지 않은 source_type: {source['source_type']}"
        
        # 텍스트 길이 확인
        if len(source["text"].strip()) < 50:
            return False, f"소스 {idx}번: 텍스트가 너무 짧습니다 (최소 50자)"
    
    return True, "검증 통과"


print("✅ 데이터 스키마 정의 완료")
print(f"   - 유효한 소스 타입: {len(VALID_SOURCE_TYPES)}가지")
print(f"   - 유효한 카테고리: {', '.join(VALID_CATEGORIES)}")

## 섹션 8: 데이터 검증 실행

In [ ]:
# ========================================
# 데이터 검증
# ========================================

print("🔍 데이터 검증 중...\n")
print("=" * 80)

validation_results = []
all_valid = True

for idx, shop_data in enumerate(collected_data, 1):
    shop_name = shop_data.get("shop_name", f"매장{idx}")
    
    print(f"\n[{idx}] {shop_name}")
    print("─" * 80)
    
    is_valid, message = validate_source_data(shop_data)
    
    if is_valid:
        print(f"✅ {message}")
        
        # 상세 정보
        num_sources = len(shop_data["sources"])
        total_chars = sum(len(s["text"]) for s in shop_data["sources"])
        source_types = [s["source_type"] for s in shop_data["sources"]]
        
        print(f"   - 카테고리: {shop_data['category']}")
        print(f"   - 소스 개수: {num_sources}개")
        print(f"   - 총 텍스트: {total_chars:,}자")
        print(f"   - 소스 타입: {', '.join(source_types)}")
        
        validation_results.append({
            "shop_name": shop_name,
            "valid": True,
            "num_sources": num_sources,
            "total_chars": total_chars
        })
    else:
        print(f"❌ {message}")
        all_valid = False
        
        validation_results.append({
            "shop_name": shop_name,
            "valid": False,
            "error": message
        })

print("\n" + "=" * 80)

if all_valid:
    print("\n✅ PHASE 2 완료: 모든 데이터 검증 통과")
    print(f"   - 총 {len(collected_data)}개 매장")
    print(f"   - 총 {sum(r['num_sources'] for r in validation_results)}개 소스")
    print(f"   - 총 {sum(r['total_chars'] for r in validation_results):,}자")
else:
    print("\n❌ PHASE 2 실패: 일부 데이터에 오류가 있습니다. 위의 메시지를 확인하세요.")

print("\n" + "=" * 80)
print("\n📌 다음 단계: PHASE 3 - 벡터 DB 인덱싱")

---

# 🗄️ PHASE 3: 벡터 DB 인덱싱 (Indexer)

검증된 소스 데이터를 청킹하고 벡터 DB에 저장합니다.

---

## 섹션 8-A: Chroma DB 완전 삭제 (선택)

⚠️ **주의**: 기존 벡터 DB를 완전히 삭제하고 싶을 때만 실행하세요.

- 주석을 해제하고 실행하면 `../chroma_db` 디렉토리 전체가 삭제됩니다.
- 모든 카테고리의 모든 데이터가 사라집니다.

In [ ]:
# ========================================
# ⚠️ 주의: 아래 주석을 해제하면 DB 전체 삭제!
# ========================================
# import shutil

# CHROMA_DB_PATH = "../chroma_db"

# shutil.rmtree(CHROMA_DB_PATH)
# print("\n✅ Chroma DB 삭제 완료!")

## 섹션 9: Chroma 벡터 DB 초기화

In [ ]:
# ========================================
# Chroma 벡터 DB 초기화
# ========================================

try:
    chroma_client = chromadb.PersistentClient(path="../chroma_db")
    
    # 카테고리별 소스 컬렉션 생성 (기존 example 컬렉션과 분리)
    fine_dining_sources = chroma_client.get_or_create_collection(
        name="fine_dining_sources",
        metadata={"hnsw:space": "cosine", "description": "파인다이닝 원본 소스"}
    )
    
    low_to_mid_price_dining_sources = chroma_client.get_or_create_collection(
        name="low_to_mid_price_dining_sources",
        metadata={"hnsw:space": "cosine", "description": "중저가 원본 소스"}
    )
    
    waiting_hotplace_sources = chroma_client.get_or_create_collection(
        name="waiting_hotplace_sources",
        metadata={"hnsw:space": "cosine", "description": "웨이팅 핫플 원본 소스"}
    )
    
    # 컬렉션 매핑
    SOURCE_COLLECTIONS = {
        "fine_dining": fine_dining_sources,
        "mid_price": low_to_mid_price_dining_sources,
        "waiting_hotplace": waiting_hotplace_sources
    }
    
    print("✅ Chroma 벡터 DB 초기화 완료 (공통 벡터 DB)")
    print(f"   - 저장 위치: ../chroma_db (shop_summary/chroma_db)")
    print(f"\n   카테고리별 컬렉션:")
    for category, collection in SOURCE_COLLECTIONS.items():
        print(f"   - {category}_sources: {collection.count()}개 청크")
    
except Exception as e:
    print(f"❌ Chroma 초기화 실패: {e}")
    chroma_client = None
    SOURCE_COLLECTIONS = {}

## 섹션 10: 청킹 및 임베딩 함수 정의

In [ ]:
# ========================================
# 청킹 설정 (source_chunking_experiment.ipynb 결과 기반)
# ========================================

CHUNKING_CONFIG = {
    # fine_dining 소스
    "michelin_review": {
        "method": "document",
        "chunk_size": None,
        "chunk_overlap": 0
    },
    "blueribbon_review": {
        "method": "document",
        "chunk_size": None,
        "chunk_overlap": 0
    },
    "chef_interview": {
        "method": "recursive",
        "chunk_size": 800,
        "chunk_overlap": 200
    },
    "course_description": {
        "method": "semantic",
        "chunk_size": 512,
        "chunk_overlap": 50
    },
    "brand_philosophy": {
        "method": "recursive",
        "chunk_size": 600,
        "chunk_overlap": 100
    },
    
    # waiting_hotplace 소스
    "signature_menu": {
        "method": "semantic",
        "chunk_size": 512,
        "chunk_overlap": 50
    },
    "atmosphere": {
        "method": "recursive",
        "chunk_size": 600,
        "chunk_overlap": 100
    },
    "popularity": {
        "method": "recursive",
        "chunk_size": 600,
        "chunk_overlap": 100
    },
    "price_value": {
        "method": "document",
        "chunk_size": None,
        "chunk_overlap": 0
    },
    "location_access": {
        "method": "recursive",
        "chunk_size": 512,
        "chunk_overlap": 50
    },
    
    # low_to_mid_price_dining 소스
    "menu_composition": {
        "method": "semantic",
        "chunk_size": 512,
        "chunk_overlap": 50
    },
    "value_proposition": {
        "method": "recursive",
        "chunk_size": 600,
        "chunk_overlap": 100
    },
    "dining_atmosphere": {
        "method": "recursive",
        "chunk_size": 600,
        "chunk_overlap": 100
    },
    "reservation_parking": {
        "method": "document",
        "chunk_size": None,
        "chunk_overlap": 0
    },
    "chef_approach": {
        "method": "recursive",
        "chunk_size": 800,
        "chunk_overlap": 200
    },
    
    # 레거시 (이전 버전 호환)
    "space_ambiance": {
        "method": "recursive",
        "chunk_size": 512,
        "chunk_overlap": 50
    },
    "tradition_technique": {
        "method": "recursive",
        "chunk_size": 600,
        "chunk_overlap": 100
    }
}


def chunk_text(text: str, source_type: str) -> List[str]:
    """소스 타입에 맞는 청킹 전략 적용"""
    config = CHUNKING_CONFIG.get(source_type, CHUNKING_CONFIG["brand_philosophy"])
    
    if config["method"] == "document":
        # 분할 없음
        return [text]
    
    elif config["method"] == "recursive":
        # LangChain RecursiveCharacterTextSplitter
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=config["chunk_size"],
            chunk_overlap=config["chunk_overlap"],
            separators=["\n\n", "\n", ". ", "? ", "! ", " ", ""],
            length_function=len
        )
        return splitter.split_text(text)
    
    elif config["method"] == "semantic":
        # 간단한 문장 단위 분할
        sentences = text.replace(". ", ".\n").replace("? ", "?\n").replace("! ", "!\n").split("\n")
        
        chunks = []
        current_chunk = []
        current_length = 0
        chunk_size = config["chunk_size"]
        chunk_overlap = config["chunk_overlap"]
        
        for sent in sentences:
            sent = sent.strip()
            if not sent:
                continue
            
            if current_length + len(sent) > chunk_size and current_chunk:
                chunks.append(" ".join(current_chunk))
                
                if chunk_overlap > 0 and current_chunk:
                    overlap_text = current_chunk[-1]
                    current_chunk = [overlap_text, sent]
                    current_length = len(overlap_text) + len(sent)
                else:
                    current_chunk = [sent]
                    current_length = len(sent)
            else:
                current_chunk.append(sent)
                current_length += len(sent)
        
        if current_chunk:
            chunks.append(" ".join(current_chunk))
        
        return chunks
    
    return [text]


def generate_embedding(text: str, model="text-embedding-004") -> List[float]:
    """텍스트를 768차원 벡터로 변환 (google.genai SDK 사용)"""
    try:
        response = client.models.embed_content(
            model=model,
            contents=text
        )
        return response.embeddings[0].values
    except Exception as e:
        print(f"❌ 임베딩 생성 실패: {e}")
        return None


def index_source_data(shop_data: Dict, collection):
    """
    한 매장의 소스 데이터를 청킹하고 벡터 DB에 저장
    
    - 같은 매장을 다시 실행하면 기존 데이터를 먼저 삭제 후 재생성
    - 고아 청크 방지를 위해 매장 단위로 완전 삭제 후 재생성
    
    Returns:
        저장된 청크 수
    """
    shop_seq = shop_data["shop_seq"]
    shop_name = shop_data["shop_name"]
    category = shop_data["category"]
    sources = shop_data["sources"]
    
    # 1. 기존 매장 데이터 삭제 (고아 청크 방지)
    try:
        existing = collection.get(
            where={"shop_seq": shop_seq},
            include=[]
        )
        if existing['ids']:
            collection.delete(ids=existing['ids'])
            print(f"   🗑️ 기존 데이터 삭제: {len(existing['ids'])}개 청크")
    except Exception as e:
        # 첫 실행 시 데이터가 없을 수 있음
        pass
    
    # 2. 새로운 데이터 추가
    total_chunks_stored = 0
    
    for source_idx, source in enumerate(sources):
        source_type = source["source_type"]
        text = source["text"]
        url = source.get("url", "")
        
        # 청킹
        chunks = chunk_text(text, source_type)
        
        # 각 청크를 벡터 DB에 저장
        for chunk_idx, chunk in enumerate(chunks):
            # 임베딩 생성
            embedding = generate_embedding(chunk)
            
            if embedding is None:
                print(f"   ⚠️ 임베딩 생성 실패: {shop_name} - {source_type} 청크 {chunk_idx}")
                continue
            
            # 고유 ID 생성
            doc_id = f"{category}_{shop_seq}_{source_idx}_{chunk_idx}"
            
            # Chroma에 저장
            collection.add(
                documents=[chunk],
                embeddings=[embedding],
                metadatas=[{
                    "shop_seq": shop_seq,
                    "shop_name": shop_name,
                    "category": category,
                    "source_type": source_type,
                    "source_url": url,
                    "chunk_index": chunk_idx,
                    "total_chunks": len(chunks),
                    "indexed_at": datetime.utcnow().isoformat()
                }],
                ids=[doc_id]
            )
            
            total_chunks_stored += 1
            
            # API Rate Limit 방지
            time.sleep(0.1)
    
    return total_chunks_stored


print("✅ 청킹 및 임베딩 함수 정의 완료")
print("   - chunk_text(): 소스 타입별 청킹")
print(f"   - 지원 소스 타입: {len(CHUNKING_CONFIG)}가지")
print("   - generate_embedding(): 768차원 임베딩 생성 (google.genai SDK)")
print("   - index_source_data(): 벡터 DB 인덱싱 (매장별 삭제 후 재생성)")

## 섹션 11: 벡터 DB 인덱싱 실행

⚠️ 주의: 이 섹션은 Vertex AI API를 호출하여 임베딩을 생성합니다. 시간이 오래 걸릴 수 있습니다.

In [ ]:
# ========================================
# 벡터 DB 인덱싱
# ========================================

if not collected_data or not all_valid:
    print("⚠️ 인덱싱할 데이터가 없거나 검증에 실패했습니다.")
else:
    print("🚀 벡터 DB 인덱싱 시작\n")
    print("=" * 80)
    print(f"\n예상 시간: 약 {len(collected_data) * 2}초 (매장당 약 2초)\n")
    
    indexing_results = []
    indexing_start_time = time.time()
    
    for idx, shop_data in enumerate(collected_data, 1):
        shop_name = shop_data["shop_name"]
        category = shop_data["category"]
        num_sources = len(shop_data["sources"])
        
        print(f"[{idx}/{len(collected_data)}] {shop_name} ({category})")
        print(f"   소스 {num_sources}개 처리 중...")
        
        # 카테고리에 맞는 컬렉션 선택
        collection = SOURCE_COLLECTIONS.get(category)
        
        if not collection:
            print(f"   ❌ 유효하지 않은 카테고리: {category}")
            indexing_results.append({
                "shop_name": shop_name,
                "category": category,
                "status": "error",
                "error": "Invalid category"
            })
            continue
        
        try:
            # 인덱싱 실행
            shop_start = time.time()
            chunks_stored = index_source_data(shop_data, collection)
            shop_elapsed = time.time() - shop_start
            
            print(f"   ✅ 완료: {chunks_stored}개 청크 저장 ({shop_elapsed:.1f}초)\n")
            
            indexing_results.append({
                "shop_name": shop_name,
                "category": category,
                "status": "success",
                "chunks_stored": chunks_stored,
                "elapsed_time": shop_elapsed
            })
            
        except Exception as e:
            print(f"   ❌ 오류: {e}\n")
            
            indexing_results.append({
                "shop_name": shop_name,
                "category": category,
                "status": "error",
                "error": str(e)
            })
    
    indexing_elapsed = time.time() - indexing_start_time
    
    print("=" * 80)
    print(f"\n✅ PHASE 3 완료: 벡터 DB 인덱싱 (총 {indexing_elapsed:.1f}초)\n")
    
    # 통계
    success_count = sum(1 for r in indexing_results if r["status"] == "success")
    error_count = len(indexing_results) - success_count
    total_chunks = sum(r.get("chunks_stored", 0) for r in indexing_results)
    
    print(f"📊 통계:")
    print(f"   - 성공: {success_count}개 매장")
    if error_count > 0:
        print(f"   - 실패: {error_count}개 매장")
    print(f"   - 총 청크: {total_chunks}개")
    print(f"   - 평균 처리 시간: {indexing_elapsed / len(collected_data):.1f}초/매장")
    
    # 카테고리별 컬렉션 크기
    print(f"\n   카테고리별 컬렉션 크기:")
    for category, collection in SOURCE_COLLECTIONS.items():
        print(f"   - {category}_sources: {collection.count()}개 청크")
    
    # 결과 저장
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    df_results = pd.DataFrame(indexing_results)
    csv_filename = f"indexing_results_{timestamp}.csv"
    df_results.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    
    print(f"\n💾 결과 저장: {csv_filename}")

---

# 🎉 파이프라인 완료!

3단계 파이프라인이 모두 완료되었습니다.

---

## 섹션 12: 전체 파이프라인 요약

In [ ]:
# ========================================
# 전체 파이프라인 요약
# ========================================

print("\n" + "=" * 80)
print("🎉 통합 소스 파이프라인 완료!")
print("=" * 80)

print("\n📊 전체 파이프라인 요약:\n")

# PHASE 1 요약
if 'collection_stats' in globals():
    phase1_success = sum(1 for s in collection_stats if s.get("status") == "success")
    phase1_sources = sum(s.get("num_sources", 0) for s in collection_stats)
    print(f"✅ PHASE 1: 소스 생성")
    print(f"   - 성공: {phase1_success}/{len(BATCH_SHOPS)}개 매장")
    print(f"   - 총 소스: {phase1_sources}개")
    print(f"   - 소요 시간: {total_elapsed:.1f}초")
    print()

# PHASE 2 요약
if 'validation_results' in globals():
    phase2_valid = sum(1 for r in validation_results if r.get("valid"))
    print(f"✅ PHASE 2: 데이터 검증")
    print(f"   - 검증 통과: {phase2_valid}/{len(validation_results)}개 매장")
    print()

# PHASE 3 요약
if 'indexing_results' in globals():
    phase3_success = sum(1 for r in indexing_results if r.get("status") == "success")
    phase3_chunks = sum(r.get("chunks_stored", 0) for r in indexing_results)
    print(f"✅ PHASE 3: 벡터 DB 인덱싱")
    print(f"   - 인덱싱 성공: {phase3_success}/{len(indexing_results)}개 매장")
    print(f"   - 총 청크: {phase3_chunks}개")
    print(f"   - 소요 시간: {indexing_elapsed:.1f}초")
    print()

# 전체 통계
if 'total_elapsed' in globals() and 'indexing_elapsed' in globals():
    total_pipeline_time = total_elapsed + indexing_elapsed
    print(f"⏱️ 전체 소요 시간: {total_pipeline_time:.1f}초 ({total_pipeline_time/60:.1f}분)")
    print()

# 비용 요약
if 'cost_estimate' in globals():
    print(f"💰 예상 비용: ${cost_estimate:.3f}")
    print()

# 벡터 DB 상태
if SOURCE_COLLECTIONS:
    print(f"🗄️ 벡터 DB 상태:")
    for category, collection in SOURCE_COLLECTIONS.items():
        print(f"   - {category}_sources: {collection.count()}개 청크")
    print()

print("=" * 80)
print("\n📌 다음 단계:")
print("   - main_rag_source.ipynb: 원본 소스 기반 RAG 실행")
print("   - main_rag_hybrid.ipynb: 하이브리드 RAG (원본 + 예시) 실행")
print("\n" + "=" * 80)

## 섹션 13: 검색 테스트 (선택)

인덱싱된 데이터가 제대로 검색되는지 테스트합니다.

In [ ]:
# ========================================
# 검색 테스트
# ========================================

# 테스트 쿼리
TEST_QUERY = "셰프의 조리 철학과 제철 식재료 활용 방법"

print(f"🔍 검색 테스트\n")
print(f"쿼리: {TEST_QUERY}\n")
print("=" * 80)

# 쿼리 임베딩 생성
query_embedding = generate_embedding(TEST_QUERY)

if query_embedding and SOURCE_COLLECTIONS:
    # 각 카테고리별로 검색
    for category, collection in SOURCE_COLLECTIONS.items():
        if collection.count() == 0:
            print(f"\n📂 {category}: 저장된 청크 없음")
            continue
        
        print(f"\n📂 {category}")
        print("─" * 80)
        
        try:
            results = collection.query(
                query_embeddings=[query_embedding],
                n_results=min(3, collection.count()),
                include=["documents", "metadatas", "distances"]
            )
            
            if results['ids'] and len(results['ids'][0]) > 0:
                for i in range(len(results['ids'][0])):
                    metadata = results['metadatas'][0][i]
                    document = results['documents'][0][i]
                    distance = results['distances'][0][i]
                    similarity = 1.0 - (distance / 2.0)
                    
                    print(f"\n   [{i+1}] {metadata['shop_name']} - {metadata['source_type']}")
                    print(f"      유사도: {similarity:.3f}")
                    print(f"      청크: {metadata['chunk_index'] + 1}/{metadata['total_chunks']}")
                    
                    preview = document[:150] + "..." if len(document) > 150 else document
                    print(f"      내용: {preview}")
            else:
                print("   검색 결과 없음")
                
        except Exception as e:
            print(f"   ❌ 검색 오류: {e}")
    
    print("\n" + "=" * 80)
else:
    print("❌ 쿼리 임베딩 생성 실패 또는 컬렉션 없음")